# 🔋 SMP 예측 모델 - CatBoost v3.19 전체 파이프라인

## 프로젝트 개요
- **목적**: 전력시장 SMP(System Marginal Price) 예측
- **모델**: CatBoost v3.19 (RTM 실시간시장용)
- **데이터**: 5년간 시간별 SMP 데이터 (43,000+ 샘플)

## 성능 목표
| Metric | Target | Achieved |
|--------|--------|----------|
| MAPE | < 7% | **5.28%** ✅ |
| R² | > 0.8 | **0.827** ✅ |

---

**Author**: RE-BMS Team  
**Last Updated**: 2025-12

# 1. 환경 설정

In [ ]:
# 필수 패키지 설치
!pip install catboost optuna pandas numpy scikit-learn matplotlib seaborn -q
print("✅ 패키지 설치 완료")

In [ ]:
# 저장소 클론
!git clone https://github.com/kiminbean/power-demand-forecast.git
%cd power-demand-forecast
print("✅ 저장소 클론 완료")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

# 시각화 설정
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")
print("✅ 라이브러리 로드 완료")

# 2. 데이터 로드 및 탐색

In [ ]:
# 데이터 로드
DATA_PATH = Path("data/smp/smp_5years_epsis.csv")
df_raw = pd.read_csv(DATA_PATH)

print(f"📊 원본 데이터 shape: {df_raw.shape}")
print(f"📅 컬럼: {df_raw.columns.tolist()}")
df_raw.head()

In [ ]:
# 시간 24:00 처리 및 datetime 변환
def fix_hour_24(ts):
    if ' 24:00' in str(ts):
        date_part = str(ts).replace(' 24:00', '')
        return pd.to_datetime(date_part) + pd.Timedelta(days=1)
    return pd.to_datetime(ts)

df = df_raw.copy()
df['datetime'] = df['timestamp'].apply(fix_hour_24)
df = df[df['smp_mainland'] > 0].copy()  # 유효 데이터만
df = df.sort_values('datetime').reset_index(drop=True)

print(f"✅ 정제된 데이터: {len(df):,}행")
print(f"📅 기간: {df['datetime'].min()} ~ {df['datetime'].max()}")
print(f"📈 SMP 범위: {df['smp_mainland'].min():.2f} ~ {df['smp_mainland'].max():.2f} 원/kWh")

In [ ]:
# 데이터 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. 전체 시계열
ax1 = axes[0, 0]
ax1.plot(df['datetime'], df['smp_mainland'], alpha=0.7, linewidth=0.5)
ax1.set_title('SMP 시계열 (5년)')
ax1.set_xlabel('날짜')
ax1.set_ylabel('SMP (원/kWh)')

# 2. 분포
ax2 = axes[0, 1]
ax2.hist(df['smp_mainland'], bins=50, edgecolor='black', alpha=0.7)
ax2.axvline(df['smp_mainland'].mean(), color='r', linestyle='--', label=f'평균: {df["smp_mainland"].mean():.1f}')
ax2.set_title('SMP 분포')
ax2.set_xlabel('SMP (원/kWh)')
ax2.legend()

# 3. 시간대별 평균
ax3 = axes[1, 0]
hourly_mean = df.groupby(df['datetime'].dt.hour)['smp_mainland'].mean()
ax3.bar(hourly_mean.index, hourly_mean.values, color='steelblue', edgecolor='black')
ax3.set_title('시간대별 평균 SMP')
ax3.set_xlabel('시간')
ax3.set_ylabel('평균 SMP (원/kWh)')
ax3.set_xticks(range(24))

# 4. 월별 평균
ax4 = axes[1, 1]
monthly_mean = df.groupby(df['datetime'].dt.month)['smp_mainland'].mean()
ax4.bar(monthly_mean.index, monthly_mean.values, color='coral', edgecolor='black')
ax4.set_title('월별 평균 SMP')
ax4.set_xlabel('월')
ax4.set_ylabel('평균 SMP (원/kWh)')

plt.tight_layout()
plt.savefig('01_data_exploration.png', dpi=150)
plt.show()

# 3. 피처 엔지니어링 (68개 피처)

In [ ]:
# 피처 이름 정의
FEATURE_NAMES = [
    # 시간 피처 (17개)
    'hour', 'hour_sin', 'hour_cos',
    'dayofweek', 'dow_sin', 'dow_cos', 'is_weekend',
    'month', 'month_sin', 'month_cos',
    'doy_sin', 'doy_cos',
    'is_summer', 'is_winter',
    'peak_morning', 'peak_evening', 'off_peak',
    # 래그 피처 (12개)
    'smp_lag1', 'smp_lag2', 'smp_lag3', 'smp_lag4', 'smp_lag5', 'smp_lag6',
    'smp_lag12', 'smp_lag24', 'smp_lag48', 'smp_lag72', 'smp_lag96', 'smp_lag168',
    # 이동 통계 (24개)
    'smp_ma6', 'smp_std6', 'smp_min6', 'smp_max6',
    'smp_ma12', 'smp_std12', 'smp_min12', 'smp_max12',
    'smp_ma24', 'smp_std24', 'smp_min24', 'smp_max24',
    'smp_ma48', 'smp_std48', 'smp_min48', 'smp_max48',
    'smp_ma72', 'smp_std72', 'smp_min72', 'smp_max72',
    'smp_ma168', 'smp_std168', 'smp_min168', 'smp_max168',
    # EMA 피처 (5개) - v3.19 신규
    'smp_ema6', 'smp_ema12', 'smp_ema24', 'smp_ema48', 'smp_ema168',
    # 차분 피처 (3개)
    'smp_diff1', 'smp_diff24', 'smp_diff168',
    # 비율 피처 (2개)
    'smp_lag1_vs_ma24', 'smp_lag1_vs_ma168',
    # 범위 피처 (2개)
    'smp_range24', 'smp_range168',
    # 모멘텀 피처 (3개) - v3.19 신규
    'smp_roc24', 'smp_roc168', 'smp_macd'
]

print(f"📊 총 피처 수: {len(FEATURE_NAMES)}개")
print(f"\n피처 카테고리:")
print(f"  - 시간 피처: 17개")
print(f"  - 래그 피처: 12개")
print(f"  - 이동 통계: 24개")
print(f"  - EMA 피처: 5개 (v3.19 신규)")
print(f"  - 차분/비율/범위: 7개")
print(f"  - 모멘텀 피처: 3개 (v3.19 신규)")

In [ ]:
def create_features(df):
    """68개 피처 생성"""
    data = df.copy()
    smp = data['smp_mainland']
    dt = data['datetime']

    # ========== 시간 피처 ==========
    data['hour'] = dt.dt.hour
    data['hour_sin'] = np.sin(2 * np.pi * data['hour'] / 24)
    data['hour_cos'] = np.cos(2 * np.pi * data['hour'] / 24)

    data['dayofweek'] = dt.dt.dayofweek
    data['dow_sin'] = np.sin(2 * np.pi * data['dayofweek'] / 7)
    data['dow_cos'] = np.cos(2 * np.pi * data['dayofweek'] / 7)
    data['is_weekend'] = (data['dayofweek'] >= 5).astype(float)

    data['month'] = dt.dt.month
    data['month_sin'] = np.sin(2 * np.pi * data['month'] / 12)
    data['month_cos'] = np.cos(2 * np.pi * data['month'] / 12)

    doy = dt.dt.dayofyear
    data['doy_sin'] = np.sin(2 * np.pi * doy / 365)
    data['doy_cos'] = np.cos(2 * np.pi * doy / 365)

    data['is_summer'] = data['month'].isin([6, 7, 8]).astype(float)
    data['is_winter'] = data['month'].isin([12, 1, 2]).astype(float)

    data['peak_morning'] = ((data['hour'] >= 9) & (data['hour'] <= 12)).astype(float)
    data['peak_evening'] = ((data['hour'] >= 17) & (data['hour'] <= 21)).astype(float)
    data['off_peak'] = ((data['hour'] >= 1) & (data['hour'] <= 6)).astype(float)

    # ========== 래그 피처 ==========
    for lag in [1, 2, 3, 4, 5, 6, 12, 24, 48, 72, 96, 168]:
        data[f'smp_lag{lag}'] = smp.shift(lag)

    # ========== 이동 통계 ==========
    for window in [6, 12, 24, 48, 72, 168]:
        data[f'smp_ma{window}'] = smp.rolling(window).mean()
        data[f'smp_std{window}'] = smp.rolling(window).std()
        data[f'smp_min{window}'] = smp.rolling(window).min()
        data[f'smp_max{window}'] = smp.rolling(window).max()

    # ========== EMA 피처 (v3.19 신규) ==========
    for span in [6, 12, 24, 48, 168]:
        data[f'smp_ema{span}'] = smp.ewm(span=span, adjust=False).mean()

    # ========== 차분 피처 ==========
    data['smp_diff1'] = smp.diff(1)
    data['smp_diff24'] = smp.diff(24)
    data['smp_diff168'] = smp.diff(168)

    # ========== 비율 피처 ==========
    data['smp_lag1_vs_ma24'] = data['smp_lag1'] / data['smp_ma24']
    data['smp_lag1_vs_ma168'] = data['smp_lag1'] / data['smp_ma168']

    # ========== 범위 피처 ==========
    data['smp_range24'] = data['smp_max24'] - data['smp_min24']
    data['smp_range168'] = data['smp_max168'] - data['smp_min168']

    # ========== 모멘텀 피처 (v3.19 신규) ==========
    data['smp_roc24'] = smp.pct_change(24)    # Rate of Change
    data['smp_roc168'] = smp.pct_change(168)
    data['smp_macd'] = data['smp_ema12'] - data['smp_ema24']  # MACD

    return data

# 피처 생성
df_features = create_features(df)
print(f"✅ 피처 생성 완료: {df_features.shape}")

In [ ]:
# 결측치 제거 및 최종 데이터셋
df_clean = df_features.dropna().copy()

X = df_clean[FEATURE_NAMES].values
y = df_clean['smp_mainland'].values

print(f"✅ 최종 데이터셋: {X.shape[0]:,}개 샘플, {X.shape[1]}개 피처")
print(f"📊 타겟 범위: {y.min():.2f} ~ {y.max():.2f} 원/kWh")
print(f"📊 타겟 평균: {y.mean():.2f} 원/kWh")

# 4. 데이터 분할 (Train / Validation / Test)

In [ ]:
# 시계열 데이터: 시간순 분할 (70% / 15% / 15%)
n = len(X)
train_end = int(n * 0.70)
val_end = int(n * 0.85)

X_train, y_train = X[:train_end], y[:train_end]
X_val, y_val = X[train_end:val_end], y[train_end:val_end]
X_test, y_test = X[val_end:], y[val_end:]

print("📊 데이터 분할:")
print(f"  - Train: {len(X_train):,}개 ({len(X_train)/n*100:.1f}%)")
print(f"  - Validation: {len(X_val):,}개 ({len(X_val)/n*100:.1f}%)")
print(f"  - Test: {len(X_test):,}개 ({len(X_test)/n*100:.1f}%)")

# 기간 확인
train_dates = df_clean.iloc[:train_end]['datetime']
val_dates = df_clean.iloc[train_end:val_end]['datetime']
test_dates = df_clean.iloc[val_end:]['datetime']

print(f"\n📅 기간:")
print(f"  - Train: {train_dates.min().date()} ~ {train_dates.max().date()}")
print(f"  - Validation: {val_dates.min().date()} ~ {val_dates.max().date()}")
print(f"  - Test: {test_dates.min().date()} ~ {test_dates.max().date()}")

# 5. 모델 학습 (CatBoost)

In [ ]:
# 최적 하이퍼파라미터 (Optuna로 튜닝된 값)
BEST_PARAMS = {
    'depth': 9,
    'learning_rate': 0.048,
    'l2_leaf_reg': 3.4,
    'bagging_temperature': 0.5,
    'random_strength': 0.003,
    'border_count': 107,
    'min_data_in_leaf': 41,
    'iterations': 2000,
    'early_stopping_rounds': 100,
    'verbose': 200,
    'random_seed': 42,
}

print("⚙️ 최적 하이퍼파라미터 (Optuna 튜닝):")
for k, v in BEST_PARAMS.items():
    print(f"  - {k}: {v}")

In [ ]:
%%time
# 모델 학습
print("🚀 CatBoost 모델 학습 시작...\n")

model = CatBoostRegressor(**BEST_PARAMS)

model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    use_best_model=True,
)

print(f"\n✅ 학습 완료!")
print(f"📊 Best iteration: {model.best_iteration_}")

# 6. 모델 평가

In [ ]:
def evaluate_model(model, X, y, set_name):
    """모델 평가 함수"""
    y_pred = model.predict(X)
    mape = mean_absolute_percentage_error(y, y_pred) * 100
    mae = mean_absolute_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    rmse = np.sqrt(np.mean((y - y_pred) ** 2))

    return {
        'set': set_name,
        'mape': mape,
        'mae': mae,
        'rmse': rmse,
        'r2': r2,
        'y_true': y,
        'y_pred': y_pred
    }

# Train / Validation / Test 평가
results = {
    'train': evaluate_model(model, X_train, y_train, 'Train'),
    'val': evaluate_model(model, X_val, y_val, 'Validation'),
    'test': evaluate_model(model, X_test, y_test, 'Test')
}

print("=" * 60)
print("📊 CatBoost v3.19 모델 평가 결과")
print("=" * 60)
print(f"{'Set':<12} {'MAPE':<10} {'MAE':<10} {'RMSE':<10} {'R²':<10}")
print("-" * 60)
for key in ['train', 'val', 'test']:
    r = results[key]
    print(f"{r['set']:<12} {r['mape']:.2f}%{'':<5} {r['mae']:.2f}{'':<5} {r['rmse']:.2f}{'':<5} {r['r2']:.4f}")
print("=" * 60)

In [ ]:
# 평가 결과 시각화
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for idx, (key, title) in enumerate([('train', 'Train'), ('val', 'Validation'), ('test', 'Test')]):
    r = results[key]

    # Scatter plot
    ax1 = axes[0, idx]
    ax1.scatter(r['y_true'], r['y_pred'], alpha=0.3, s=10)
    ax1.plot([r['y_true'].min(), r['y_true'].max()],
             [r['y_true'].min(), r['y_true'].max()], 'r--', lw=2)
    ax1.set_title(f"{title} (R² = {r['r2']:.4f})")
    ax1.set_xlabel('Actual SMP')
    ax1.set_ylabel('Predicted SMP')

    # Error distribution
    ax2 = axes[1, idx]
    errors = r['y_true'] - r['y_pred']
    ax2.hist(errors, bins=50, edgecolor='black', alpha=0.7)
    ax2.axvline(0, color='r', linestyle='--', lw=2)
    ax2.set_title(f"{title} Error (MAPE = {r['mape']:.2f}%)")
    ax2.set_xlabel('Error (won/kWh)')

plt.tight_layout()
plt.savefig('02_model_evaluation.png', dpi=150)
plt.show()

# 7. 피처 중요도 분석

In [ ]:
# 피처 중요도
feature_importance = model.get_feature_importance()
importance_df = pd.DataFrame({
    'feature': FEATURE_NAMES,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

# Top 20 시각화
plt.figure(figsize=(10, 8))
top_n = 20
plt.barh(range(top_n), importance_df['importance'].head(top_n).values[::-1], color='steelblue')
plt.yticks(range(top_n), importance_df['feature'].head(top_n).values[::-1])
plt.xlabel('Importance')
plt.title(f'Top {top_n} Feature Importance')
plt.tight_layout()
plt.savefig('03_feature_importance.png', dpi=150)
plt.show()

print("\n📊 Top 10 피처:")
print(importance_df.head(10).to_string(index=False))

# 8. 시간대별 성능 분석

In [ ]:
# 테스트셋 시간대별 분석
test_df = df_clean.iloc[val_end:].copy()
test_df['predicted'] = results['test']['y_pred']
test_df['error'] = test_df['smp_mainland'] - test_df['predicted']
test_df['abs_pct_error'] = np.abs(test_df['error'] / test_df['smp_mainland']) * 100

# 시간대별 MAPE
hourly_mape = test_df.groupby('hour')['abs_pct_error'].mean()

plt.figure(figsize=(12, 5))
colors = ['green' if x < results['test']['mape'] else 'coral' for x in hourly_mape.values]
plt.bar(hourly_mape.index, hourly_mape.values, color=colors, edgecolor='black')
plt.axhline(results['test']['mape'], color='r', linestyle='--', lw=2,
            label=f'Overall MAPE: {results["test"]["mape"]:.2f}%')
plt.xlabel('Hour')
plt.ylabel('MAPE (%)')
plt.title('시간대별 MAPE (Test Set)')
plt.xticks(range(24))
plt.legend()
plt.tight_layout()
plt.savefig('04_hourly_mape.png', dpi=150)
plt.show()

print("\n📊 시간대별 MAPE:")
print(hourly_mape.round(2).to_string())

# 9. 예측 시계열 비교

In [ ]:
# 테스트셋 마지막 1주일 예측 비교
n_plot = 168  # 1주일

plt.figure(figsize=(14, 6))
plt.plot(range(n_plot), results['test']['y_true'][-n_plot:],
         label='Actual', alpha=0.8, linewidth=2)
plt.plot(range(n_plot), results['test']['y_pred'][-n_plot:],
         label='Predicted', alpha=0.8, linewidth=2)
plt.fill_between(range(n_plot),
                 results['test']['y_pred'][-n_plot:] * 0.9,
                 results['test']['y_pred'][-n_plot:] * 1.1,
                 alpha=0.2, label='±10% Band')
plt.xlabel('Hour')
plt.ylabel('SMP (won/kWh)')
plt.title('Test Set - Last 168 Hours (1 Week) Prediction')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('05_prediction_timeseries.png', dpi=150)
plt.show()

# 10. 모델 저장

In [ ]:
# 모델 저장
MODEL_SAVE_DIR = Path("models/catboost_colab")
MODEL_SAVE_DIR.mkdir(parents=True, exist_ok=True)

# CatBoost 모델 저장
model.save_model(str(MODEL_SAVE_DIR / "catboost_model.cbm"))

# 메트릭 저장
metrics = {
    'train_mape': results['train']['mape'],
    'train_r2': results['train']['r2'],
    'val_mape': results['val']['mape'],
    'val_r2': results['val']['r2'],
    'test_mape': results['test']['mape'],
    'test_r2': results['test']['r2'],
    'test_mae': results['test']['mae'],
    'n_samples': len(X),
    'n_features': len(FEATURE_NAMES),
    'best_params': BEST_PARAMS,
    'feature_names': FEATURE_NAMES,
}

with open(MODEL_SAVE_DIR / "metrics.json", 'w') as f:
    json.dump(metrics, f, indent=2, default=str)

print(f"✅ 모델 저장 완료: {MODEL_SAVE_DIR}")
print(f"  - catboost_model.cbm")
print(f"  - metrics.json")

# 11. 최종 요약

In [ ]:
print("="*70)
print("🔋 SMP 예측 모델 - CatBoost v3.19 최종 요약")
print("="*70)
print()
print("📊 데이터:")
print(f"   - 총 샘플: {len(X):,}개")
print(f"   - 피처 수: {len(FEATURE_NAMES)}개")
print(f"   - 기간: 약 5년 (시간별)")
print()
print("📈 모델 성능:")
print(f"   ┌{'─'*20}┬{'─'*12}┬{'─'*12}┬{'─'*12}┐")
print(f"   │ {'Set':<18} │ {'MAPE':<10} │ {'R²':<10} │ {'MAE':<10} │")
print(f"   ├{'─'*20}┼{'─'*12}┼{'─'*12}┼{'─'*12}┤")
print(f"   │ {'Train':<18} │ {results['train']['mape']:>8.2f}% │ {results['train']['r2']:>10.4f} │ {results['train']['mae']:>10.2f} │")
print(f"   │ {'Validation':<18} │ {results['val']['mape']:>8.2f}% │ {results['val']['r2']:>10.4f} │ {results['val']['mae']:>10.2f} │")
print(f"   │ {'Test':<18} │ {results['test']['mape']:>8.2f}% │ {results['test']['r2']:>10.4f} │ {results['test']['mae']:>10.2f} │")
print(f"   └{'─'*20}┴{'─'*12}┴{'─'*12}┴{'─'*12}┘")
print()
print("🎯 목표 달성:")
mape_target = 7.0
r2_target = 0.8
test_mape = results['test']['mape']
test_r2 = results['test']['r2']
print(f"   - MAPE: {test_mape:.2f}% {'✅ 달성' if test_mape < mape_target else '❌ 미달'} (목표: < {mape_target}%)")
print(f"   - R²: {test_r2:.4f} {'✅ 달성' if test_r2 > r2_target else '❌ 미달'} (목표: > {r2_target})")
print()
print("📌 Top 5 피처:")
for i, row in importance_df.head(5).iterrows():
    print(f"   {importance_df.index.get_loc(i)+1}. {row['feature']}: {row['importance']:.2f}")
print()
print("🔄 BiLSTM v3.2 대비:")
bilstm_mape = 7.17
bilstm_r2 = 0.77
print(f"   - MAPE: {test_mape:.2f}% vs {bilstm_mape}% ({bilstm_mape - test_mape:.2f}%p 개선)")
print(f"   - R²: {test_r2:.4f} vs {bilstm_r2:.2f} ({test_r2 - bilstm_r2:.4f} 개선)")
print()
print("💡 용도:")
print("   - RTM (실시간시장): CatBoost v3.19 - 단일 스텝 예측")
print("   - DAM (하루전시장): BiLSTM v3.2 - 24시간 시퀀스 예측")
print("="*70)